# Обработка XML

### Задание 1

Для доступного по ссылке https://stepik.org/media/attachments/lesson/245678/map1.osm фрагмента карты посчитайте, сколько node имеет хотя бы один вложенный тэг tag, а сколько - не имеют. В качестве ответа введите два числа, разделённых пробелом.

In [ ]:
import requests
from bs4 import BeautifulSoup
osm = requests.get('https://stepik.org/media/attachments/lesson/245678/map1.osm').text
soup = BeautifulSoup(osm, 'lxml-xml')

In [ ]:
without_tag = 0
with_tag = 0

for node in soup.find_all('node'):
    if node.find('tag') is None:
        without_tag += 1
    else:
        with_tag += 1
print(f'С тегом: {with_tag}\nБез тега: {without_tag}')

С тегом: 316
Без тега: 6166


### Задание 2

Вася решил открыть АЗС (заправку). Чтобы оценить уровень конкуренции он хочет изучить количество заправок в интересующем его районе. Вася скачал интересующий его кусок карты OSM и хочет посчитать, сколько на нём отмечено точечных объектов (node), являющихся заправкой. В качестве ответа вам необходимо вывести одно число - количество АЗС.

In [ ]:
osm = requests.get('https://stepik.org/media/attachments/lesson/245681/map2.osm').text
soup = BeautifulSoup(osm, 'lxml-xml')

In [ ]:
cnt = 0
for node in soup.find_all('node'):
    if node.find('tag', k='amenity', v='fuel'):
        cnt += 1
print(cnt)

15


### Задание 3

Вася, открывший заправку в прошлом уроке, разорился. Конкуренция оказалась слишком большой. Вася предполагает, что это произошло от того, что теги заправки могут быть не только на точке, но и на каком-то контуре. Определите, сколько заправок на самом деле (не только обозначенных точкой) есть на фрагменте карты

In [ ]:
osm = requests.get('https://stepik.org/media/attachments/lesson/245681/map2.osm').text
soup = BeautifulSoup(osm, 'lxml-xml')
print(len(soup.find_all('tag', k='amenity', v='fuel')))

27


# Работа с OSM

### Задание

Экспортируйте с OSM часть карты, на которой находитесь вы (или любую другую интересную вам).

Посчитайте на карте объекты какого-либо типа. 

In [ ]:
# Для выполнения задания была экспортирована карта с Новой Голландией и 
# территорией вокруг нее: https://www.openstreetmap.org/export#map=16/59.9302/30.2915

# И посчитано кол-во зданий разного типа на фрагменте

with open('map.osm', 'r', encoding='utf8') as f:
    xml = BeautifulSoup(f.read(), 'lxml-xml')

### Считаем кол-во зданий разного типа

In [ ]:
build_all = xml.find_all('tag', k='building')
build_live = xml.find_all('tag', k='building', v=['apartments', 'bungalow',
                                                  'cabin', 'detached', 'ger',
                                                  'dormitory', 'farm', 'hotel',
                                                  'house', 'houseboat', 'residential',
                                                  'semidetached_house', 'static_caravan',
                                                  'terrace'])

build_com = xml.find_all('tag', k='building', v=['commercial', 'industrial',
                                                 'kiosk', 'office', 'retail',
                                                 'supermarket', 'warehouse'])

build_rel = xml.find_all('tag', k='building', v=['cathedral', 'chapel', 'church',
                                                 'mosque', 'religious', 'shrine',
                                                 'synagogue', 'temple'])

build_publ = xml.find_all('tag', k='building', v=['bakehouse', 'civic',
                                                  'fire_station', 'government',
                                                  'hospital', 'kindergarten', 'public',
                                                  'school', 'toilets', 'train_station',
                                                  'transportation', 'university'])

build_sport = xml.find_all('tag', k='building', v=['grandstand', 'pavilion', 'riding_hall',
                                                 'sports_hall', 'stadium'])

build_car = xml.find_all('tag', k='building', v=['carport', 'garage', 'garages',
                                                 'parking'])

build_tech = xml.find_all('tag', k='building', v=['digester', 'service', 'transformer_tower',
                                                 'water_tower'])

build_oth = xml.find_all('tag', k='building', v=['bunker', 'bridge', 'construction',
                                                 'roof', 'ruins'])

build_non = xml.find_all('tag', k='building', v=['yes'])

In [4]:
build = {'Нет типа': len(build_non),
         'Коммерческие': len(build_com),
         'Технические': len(build_tech),
         'Жилые': len(build_live),
         'Общественные': len(build_publ),
         'Гаражи/Парковки': len(build_car),
         'Другие': len(build_oth),
         'Религиозные': len(build_rel),
         'Стадионы/Спортзалы': len(build_sport)
         }
print(f'Общее кол-во зданий: {len(build_all)}\n')
print('{: <20}  {: >3}'.format('Тип здания', 'Кол-во'))
for k,v in build.items():
    print('{: <20} | {: >5}'.format(k, v))

Общее кол-во зданий: 389

Тип здания            Кол-во
Нет типа             |   347
Коммерческие         |    14
Технические          |    11
Жилые                |     6
Общественные         |     6
Гаражи/Парковки      |     4
Другие               |     1
Религиозные          |     0
Стадионы/Спортзалы   |     0


### Считаем кол-во категорий у зданий

In [ ]:
from collections import Counter

tags = {}
for i in range(len(build_all)):
    t = len([build_all[i]['v']])
    tags[f'Здание №{i}'] = t

val = tags.values()
count_tag = dict(Counter(val))

In [78]:
print(f'Общее кол-во зданий: {len(build_all)}\n')
print('{: <12}  {: >14}'.format('Кол-во категорий', 'Кол-во зданий'))
for k,v in count_tag.items():
    print('{: >16} | {: >13}'.format(k, v))

Общее кол-во зданий: 389

Кол-во категорий   Кол-во зданий
               1 |           389


# Продолжаем работу с BeautifulSoup

### Задание 1

Вашей программе на вход подаются две строки, содержащие url двух документов A и B. Выведите Yes, если из A в B можно перейти за два перехода, иначе выведите No.

In [ ]:
def search_link(l):
    soup = BeautifulSoup(requests.get(l).text, 'html5lib')
    return [a['href'] for a in soup.find_all('a', href=True)]

In [ ]:
# test 1
start = 'https://stepic.org/media/attachments/lesson/24472/sample0.html'
end = 'https://stepic.org/media/attachments/lesson/24472/sample2.html'

url = search_link(start)
if end in url:
    print('No')
else:
    for i in url:
        url_2 = search_link(i)
        if end in url_2:
            print('Yes')

Yes


In [ ]:
# test 2
start = 'https://stepic.org/media/attachments/lesson/24472/sample0.html'
end = 'https://stepic.org/media/attachments/lesson/24472/sample1.html'

url = search_link(start)
if end in url:
    print('No')
else:
    for i in url:
        url_2 = search_link(i)
        if end in url_2:
            print('Yes')
            break

No


### Задание 2

Вашей программе на вход подается ссылка на HTML файл. Вам необходимо скачать этот файл, затем найти в нем все ссылки вида <a ... href="..." ... > и вывести список сайтов, на которые есть ссылка.

In [ ]:
import re
link = """<a href="http://stepic.org/courses">
<a href='https://stepic.org'>
<a href='http://neerc.ifmo.ru:1345'>
<a href="ftp://mail.ru/distib" >
<a href="ya.ru">
<a href="www.ya.ru">
<a href="../skip_relative_links">"""

soup_html = BeautifulSoup(link, 'html5lib')

In [ ]:
domain = []
for a in soup_html('a', href=True):
    d = re.findall('(?:www\.)*((?:[\w]+\.)+\w+)/*', a['href'])
    if len(d) > 0 and d[0] not in domain:
        domain.append(d[0])
for i in sorted(domain):
    print(i)

mail.ru
neerc.ifmo.ru
stepic.org
ya.ru
